In [27]:
import os
os.chdir('c:\\Users\\utilisateur\\Documents\\GitHub\\ML_Project1\\ML_project1')
         
import sys
sys.path.append(os.getcwd())

In [28]:
import numpy as np
from helpers import *

In [101]:
"""
import pandas as pd

# Load your large dataset
file_path = 'dataset\\x_train.csv'
df = pd.read_csv(file_path)

# Define the number of rows you want in each smaller file
rows_per_file = 50000  # You can change this based on the file size

# Split the dataset
for i in range(0, len(df), rows_per_file):
    # Create a smaller dataset slice
    df_subset = df[i:i+rows_per_file]
    
    # Save this slice to a new CSV file
    output_file = f'small_dataset_part_{i//rows_per_file + 1}.csv'
    df_subset.to_csv(output_file, index=False)
    print(f'Created {output_file}')
"""

"\nimport pandas as pd\n\n# Load your large dataset\nfile_path = 'dataset\\x_train.csv'\ndf = pd.read_csv(file_path)\n\n# Define the number of rows you want in each smaller file\nrows_per_file = 50000  # You can change this based on the file size\n\n# Split the dataset\nfor i in range(0, len(df), rows_per_file):\n    # Create a smaller dataset slice\n    df_subset = df[i:i+rows_per_file]\n    \n    # Save this slice to a new CSV file\n    output_file = f'small_dataset_part_{i//rows_per_file + 1}.csv'\n    df_subset.to_csv(output_file, index=False)\n    print(f'Created {output_file}')\n"

## Load the data 

In [44]:
# Load dataset
file_path = 'dataset\\'
x_train, x_test, y_train, train_ids, test_ids = load_csv_data(file_path)

removal_log = []


In [45]:
removed_missing_values = []
removed_variance = []
removed_correlation = []

In [46]:
def log_and_print_removals(headers, removed_indices, step_name, reason):
    """
    Logs and prints the features removed at each step.

    Args:
        headers (list): The list of feature names.
        removed_indices (list): Indices of features that were removed at the current step.
        step_name (str): The name of the step where the features were removed.
        reason (str): The reason why the features were removed.
    """
    removed_features = [headers[i] for i in removed_indices]
    for feature in removed_features:
        removal_log.append(f"{feature} removed at {step_name}: {reason}")

    print(f"\nStep {step_name} - Reason: {reason}:")
    print(f"Removed {len(removed_features)} features: {removed_features}")

In [47]:
# Function to print kept/removed features after each step
def print_feature_info(headers, kept_indices, step_name):
    """
    Prints the number of features that were kept and removed after each step, 
    along with the respective feature names.

    Args:
        headers (list): The list of feature names.
        kept_indices (list): Indices of features that were kept after the step.
        step_name (str): The name of the step for which this information is printed.
    """
    kept_features = [headers[i] for i in kept_indices]
    removed_features = [headers[i] for i in range(len(headers)) if i not in kept_indices]
    
    print(f"\nStep {step_name}:")
    print(f"Kept {len(kept_features)} features: {kept_features}")
    print(f"Removed {len(removed_features)} features: {removed_features}")

In [48]:
def extract_headers(csv_file_path):
    """
    This function extracts the headers (feature names) from the given CSV file.
    
    Args:
        csv_file_path (str): Path to the CSV file from which to extract headers.
        
    Returns:
        list: A list of headers (feature names), excluding the 'Id' column.
    """
    with open(csv_file_path, 'r') as f:
        reader = csv.reader(f)
        headers = next(reader)  # Read the first row as headers
    return headers[1:]  # Exclude the first column header as it is 'Id'

In [49]:
# Extract headers from x_train.csv
x_train_headers = extract_headers("dataset\\x_train.csv")

In [50]:
# Initialize counters and lists
binary_count = 0
continuous_count = 0
categorical_count = 0

binary_features = []
continuous_features = []
categorical_features = []

# Threshold for distinguishing categorical from continuous
categorical_threshold = 10

# Analyze each feature column-wise
for col_idx in range(x_train.shape[1]):
    column_data = x_train[:, col_idx]
    
    # Remove missing or empty values to avoid counting them
    column_data = column_data[~np.isnan(column_data)]  # Remove NaN values

    # Get unique values
    unique_values = np.unique(column_data)
    unique_count = len(unique_values)

    # Binary feature: 2 unique values
    if unique_count == 2 and set(unique_values) == {0, 1}:
        binary_count += 1
        binary_features.append(x_train_headers[col_idx])
    
    # Continuous feature: more than categorical_threshold unique values
    elif unique_count > categorical_threshold:
        continuous_count += 1
        continuous_features.append(x_train_headers[col_idx])
    
    # Categorical feature: 3-10 unique values (or another condition you define)
    else:
        categorical_count += 1
        categorical_features.append(x_train_headers[col_idx])

# Output the counts and the feature types
print(f"Binary features: {binary_count}")
print(f"Continuous features: {continuous_count}")
print(f"Categorical features: {categorical_count}")

# Optionally, display the lists of features by type
print("Binary features:", binary_features)
print("Continuous features:", continuous_features)
print("Categorical features:", categorical_features)

Binary features: 4
Continuous features: 102
Categorical features: 215
Binary features: ['_FRTRESP', '_VEGRESP', '_FRT16', '_VEG23']
Continuous features: ['_STATE', 'FMONTH', 'IDATE', 'IMONTH', 'IDAY', 'SEQNO', '_PSU', 'NUMADULT', 'NUMMEN', 'NUMWOMEN', 'HHADULT', 'PHYSHLTH', 'MENTHLTH', 'POORHLTH', 'DIABAGE2', 'CHILDREN', 'WEIGHT2', 'HEIGHT3', 'ALCDAY5', 'AVEDRNK2', 'DRNK3GE5', 'MAXDRNKS', 'FRUITJU1', 'FRUIT1', 'FVBEANS', 'FVGREEN', 'FVORANG', 'VEGETAB1', 'EXRACT11', 'EXEROFT1', 'EXERHMM1', 'EXRACT21', 'EXEROFT2', 'EXERHMM2', 'STRENGTH', 'JOINPAIN', 'FLSHTMY2', 'IMFVPLAC', 'HIVTSTD3', 'WHRTST10', 'BLDSUGAR', 'FEETCHK2', 'DOCTDIAB', 'CHKHEMO3', 'FEETCHK', 'CRGVREL1', 'CRGVPRB1', 'LONGWTCH', 'ASTHMAGE', 'ASDRVIST', 'ASRCHKUP', 'ASACTLIM', 'SCNTWRK1', 'SCNTLWK1', 'ADPLEASR', 'ADDOWN', 'ADSLEEP', 'ADENERGY', 'ADEAT1', 'ADFAIL', 'ADTHINK', 'ADMOVE', '_STSTR', '_STRWT', '_RAWRAKE', '_WT2RAKE', '_CLLCPWT', '_DUALCOR', '_LLCPWT', '_AGEG5YR', '_AGE80', 'HTIN4', 'HTM4', 'WTKG3', '_BMI5', 'DROCDY3

## Step 1 : Preprocessing 

### A. Remove features with too many missing values

In [52]:
def remove_missing_values(headers, data, threshold=0.25):
    """
    Removes features (columns) from the dataset that have more than the specified
    percentage of missing values.

    Args:
        headers (list): The list of feature names.
        data (np.array): The dataset (numpy array) with features as columns.
        threshold (float): The maximum allowed percentage of missing values (default is 25%).

    Returns:
        tuple: A tuple containing:
            - valid_columns (list): The indices of the features that were kept.
            - filtered_data (np.array): The filtered dataset with only the valid columns.
    """
    # Find the percentage of missing values for each column
    missing_values = np.isnan(data).mean(axis=0)
    valid_columns = np.where(missing_values <= threshold)[0]
    removed_columns = np.where(missing_values > threshold)[0]

    # Store the removed features
    removed_features_names = [headers[i] for i in removed_columns]
    removed_missing_values.extend(removed_features_names)
    
    # Log removed features
    log_and_print_removals(headers, removed_columns, "1: Remove Missing Values", "Too many missing values")
    
    # Filter out the columns with too many missing values
    filtered_data = data[:, valid_columns]
    return valid_columns, filtered_data


In [53]:
removed_missing_values = []  # Initialize the list to store removed features

# Apply the function to remove columns with too many missing values
valid_columns, x_train_filtered = remove_missing_values(x_train_headers, x_train)

# Update headers to match the filtered dataset
x_train_headers_filtered = [x_train_headers[i] for i in valid_columns]

# Display results
print("Filtered dataset shape:", x_train_filtered.shape)
print("Remaining features:", x_train_headers_filtered)
print("Removed features due to missing values:", removed_missing_values)


Step 1: Remove Missing Values - Reason: Too many missing values:
Removed 178 features: ['CTELENUM', 'PVTRESD1', 'COLGHOUS', 'STATERES', 'CELLFON3', 'LADULT', 'NUMADULT', 'NUMMEN', 'NUMWOMEN', 'CTELNUM1', 'CELLFON2', 'CADULT', 'PVTRESD2', 'CCLGHOUS', 'CSTATE', 'LANDLINE', 'HHADULT', 'POORHLTH', 'BPMEDS', 'ASTHNOW', 'DIABAGE2', 'NUMHHOL2', 'NUMPHON2', 'CPDEMO1', 'PREGNANT', 'SMOKDAY2', 'STOPSMK2', 'LASTSMK2', 'AVEDRNK2', 'DRNK3GE5', 'MAXDRNKS', 'EXRACT11', 'EXEROFT1', 'EXERHMM1', 'EXRACT21', 'EXEROFT2', 'EXERHMM2', 'LMTJOIN3', 'ARTHDIS2', 'ARTHSOCL', 'JOINPAIN', 'FLSHTMY2', 'IMFVPLAC', 'HIVTSTD3', 'WHRTST10', 'PDIABTST', 'PREDIAB1', 'INSULIN', 'BLDSUGAR', 'FEETCHK2', 'DOCTDIAB', 'CHKHEMO3', 'FEETCHK', 'EYEEXAM', 'DIABEYE', 'DIABEDU', 'CAREGIV1', 'CRGVREL1', 'CRGVLNG1', 'CRGVHRS1', 'CRGVPRB1', 'CRGVPERS', 'CRGVHOUS', 'CRGVMST2', 'CRGVEXPT', 'VIDFCLT2', 'VIREDIF3', 'VIPRFVS2', 'VINOCRE2', 'VIEYEXM2', 'VIINSUR2', 'VICTRCT4', 'VIGLUMA2', 'VIMACDG2', 'CIMEMLOS', 'CDHOUSE', 'CDASSIST', 'CDHEL

In [54]:
#Check that No Columns Exceed the Missing Value Threshold
remaining_missing_percentage = np.isnan(x_train_filtered).mean(axis=0)
if np.all(remaining_missing_percentage <= 0.25):
    print("Success: All remaining columns meet the missing value threshold.")
else:
    print("Warning: Some columns still exceed the missing value threshold.")

Success: All remaining columns meet the missing value threshold.


### B. Transform categorical data into binary data by hot-encoding

In [56]:
def one_hot_encode(data, headers, categorical_columns):
    """
    Transforms categorical features into binary (one-hot encoded) features.

    Args:
        data (np.array): The dataset as a numpy array.
        headers (list): List of feature names corresponding to data columns.
        categorical_columns (list): List of indices of categorical features in the dataset.

    Returns:
        np.array: Updated dataset with one-hot encoded features.
        list: Updated headers with new binary feature names.
    """
    updated_data = []
    updated_headers = []

    for col_idx in range(data.shape[1]):
        if col_idx in categorical_columns:
            # Get unique categories for this feature
            unique_categories = np.unique(data[~np.isnan(data[:, col_idx]), col_idx])
            
            # Create binary columns for each category
            for category in unique_categories:
                binary_column = (data[:, col_idx] == category).astype(int)
                updated_data.append(binary_column)
                
                # Add new binary feature name to headers
                updated_headers.append(f"{headers[col_idx]}_{category}_encoded")
        else:
            # For non-categorical columns, add them directly
            updated_data.append(data[:, col_idx])
            updated_headers.append(headers[col_idx])

    # Stack the updated data columns horizontally
    updated_data = np.column_stack(updated_data)
    return updated_data, updated_headers

# Apply one-hot encoding on categorical features in x_train
categorical_columns_filtered = [i for i, header in enumerate(x_train_headers_filtered) if header in categorical_features]

x_train_encoded, x_train_encoded_headers = one_hot_encode(x_train_filtered, x_train_headers_filtered, categorical_columns_filtered)

# Step 3: Display results
print("Encoded dataset shape:", x_train_encoded.shape)
print("Encoded headers:", x_train_encoded_headers)

Encoded dataset shape: (328135, 463)
Encoded headers: ['_STATE', 'FMONTH', 'IDATE', 'IMONTH', 'IDAY', 'IYEAR_2015.0_encoded', 'IYEAR_2016.0_encoded', 'DISPCODE_1100.0_encoded', 'DISPCODE_1200.0_encoded', 'SEQNO', '_PSU', 'GENHLTH_1.0_encoded', 'GENHLTH_2.0_encoded', 'GENHLTH_3.0_encoded', 'GENHLTH_4.0_encoded', 'GENHLTH_5.0_encoded', 'GENHLTH_7.0_encoded', 'GENHLTH_9.0_encoded', 'PHYSHLTH', 'MENTHLTH', 'HLTHPLN1_1.0_encoded', 'HLTHPLN1_2.0_encoded', 'HLTHPLN1_7.0_encoded', 'HLTHPLN1_9.0_encoded', 'PERSDOC2_1.0_encoded', 'PERSDOC2_2.0_encoded', 'PERSDOC2_3.0_encoded', 'PERSDOC2_7.0_encoded', 'PERSDOC2_9.0_encoded', 'MEDCOST_1.0_encoded', 'MEDCOST_2.0_encoded', 'MEDCOST_7.0_encoded', 'MEDCOST_9.0_encoded', 'CHECKUP1_1.0_encoded', 'CHECKUP1_2.0_encoded', 'CHECKUP1_3.0_encoded', 'CHECKUP1_4.0_encoded', 'CHECKUP1_7.0_encoded', 'CHECKUP1_8.0_encoded', 'CHECKUP1_9.0_encoded', 'BPHIGH4_1.0_encoded', 'BPHIGH4_2.0_encoded', 'BPHIGH4_3.0_encoded', 'BPHIGH4_4.0_encoded', 'BPHIGH4_7.0_encoded', 'BP

In [57]:
def verify_one_hot_encoding(data, binary_columns_indices):
    """
    Verifies that all binary columns contain only 0s and 1s, and that all columns
    in the dataset are either binary or continuous.

    Args:
        data (np.array): The dataset after one-hot encoding.
        binary_columns_indices (list): List of indices of columns that should be binary.

    Returns:
        bool: True if all checks pass, False otherwise.
    """
    # Check binary columns
    binary_check_passed = True
    for col in binary_columns_indices:
        unique_values = np.unique(data[:, col])
        if not np.array_equal(unique_values, [0, 1]) and not np.array_equal(unique_values, [1, 0]):
            print(f"Check failed: Column {col} has values other than 0 and 1.")
            binary_check_passed = False

    # Check non-binary columns are continuous
    continuous_check_passed = True
    for col in range(data.shape[1]):
        if col not in binary_columns_indices:
            unique_values = np.unique(data[:, col])
            if len(unique_values) < 10:  # Continuous data should have a broad range of unique values
                print(f"Check failed: Column {col} appears to have low unique values, suggesting it may not be continuous.")
                continuous_check_passed = False

    # Summary
    if binary_check_passed and continuous_check_passed:
        print("All checks passed: The dataset has only binary or continuous features.")
        return True
    else:
        print("Some checks failed. Please review the warnings above.")
        return False

# Example usage after one-hot encoding
# Get the indices for original binary features
original_binary_indices = [i for i, header in enumerate(x_train_encoded_headers) if header in binary_features]

# Get the indices for one-hot encoded features
one_hot_encoded_indices = [i for i, header in enumerate(x_train_encoded_headers) if "_encoded" in header]

# Concatenate both lists to form the full list of binary column indices
binary_columns_indices = original_binary_indices + one_hot_encoded_indices

# Now use the combined binary_columns_indices to verify the dataset
verify_one_hot_encoding(x_train_encoded, binary_columns_indices)

All checks passed: The dataset has only binary or continuous features.


True

### B. Imputation methods

Mean Imputation (nice for normal distribution assumption)

In [60]:
def mean_imputation(data, binary_columns):
    """
    Imputes missing values in the dataset by replacing them with the mean of each column.
    For binary columns, the mean is rounded to 0 or 1.

    Args:
        data (np.array): The dataset with missing values (numpy array with NaN for missing values).
        binary_columns (list): List of indices representing binary columns.

    Returns:
        np.array: The dataset with missing values replaced by column means or rounded means for binary features.
    """
    col_means = np.nanmean(data, axis=0)
    inds = np.where(np.isnan(data))
    
    # For binary columns, round the mean to 0 or 1
    for col in binary_columns:
        col_means[col] = round(col_means[col])

    # Replace NaNs with the appropriate mean (rounded for binary columns)
    data[inds] = np.take(col_means, inds[1])
    
    return data

In [58]:
# Identify binary and continuous columns
binary_columns_indices = [i for i, header in enumerate(x_train_encoded_headers) if "_encoded" in header or header in binary_features]
continuous_columns_indices = [i for i in range(x_train_encoded.shape[1]) if i not in binary_columns_indices]

In [62]:
# Apply Mean Imputation on the encoded dataset
x_train_mean_imputed = mean_imputation(x_train_encoded.copy(), binary_columns=binary_columns_indices)

In [63]:
if np.isnan(x_train_mean_imputed).any():
    print("Warning: There are still missing values in the dataset after mean imputation.")
else:
    print("Success: No missing values in the dataset.")

Success: No missing values in the dataset.


In [64]:
binary_columns_valid = True
for col in binary_columns_indices:
    unique_values = np.unique(x_train_mean_imputed[:, col])
    if not np.array_equal(unique_values, [0, 1]) and not np.array_equal(unique_values, [1, 0]):
        print(f"Check failed: Column {col} has values other than 0 and 1.")
        binary_columns_valid = False

if binary_columns_valid:
    print("Success: All binary columns contain only 0s and 1s after mean imputation.")

Success: All binary columns contain only 0s and 1s after mean imputation.


In [65]:
continuous_columns_valid = True
for col in continuous_columns_indices:
    mean_value = np.nanmean(x_train_encoded[:, col])  # Original mean (ignoring NaNs)
    imputed_mean = np.mean(x_train_mean_imputed[:, col])  # Mean after imputation
    if np.isclose(mean_value, imputed_mean, rtol=0.1):  # Allow slight tolerance due to rounding differences
        continue
    else:
        print(f"Check failed: Column {col} imputed mean differs significantly from expected mean.")
        continuous_columns_valid = False

if continuous_columns_valid:
    print("Success: All continuous columns imputed correctly with mean values.")

Success: All continuous columns imputed correctly with mean values.


In [66]:
original_shape = x_train_encoded.shape
imputed_shape = x_train_mean_imputed.shape
if original_shape == imputed_shape:
    print("Shape consistency check passed.")
else:
    print(f"Warning: Shape changed from {original_shape} to {imputed_shape} after imputation.")

Shape consistency check passed.


## Step 2 : Variance Thresholding

In [71]:
def variance_thresholding(headers, data, threshold=0.01):
    """
    Removes features that have variance below a specified threshold.

    Args:
        headers (list): The list of feature names.
        data (np.array): The dataset (numpy array) with features as columns.
        threshold (float): The minimum variance threshold (default is 1%).

    Returns:
        tuple: A tuple containing:
            - valid_columns (list): The indices of the features that were kept.
            - filtered_data (np.array): The filtered dataset with only the valid columns.
    """
    # Compute variance for each column
    variances = np.nanvar(data, axis=0)
    valid_columns = np.where(variances >= threshold)[0]
    removed_columns = np.where(variances < threshold)[0]

    # Store the removed features
    removed_features_names = [headers[i] for i in removed_columns]
    removed_variance.extend(removed_features_names)
    
    # Log removed features
    log_and_print_removals(headers, removed_columns, "2: Variance Thresholding", "Low variance")
    
    # Keep only columns with variance above the threshold
    filtered_data = data[:, valid_columns]
    return valid_columns, filtered_data

In [72]:
# Apply the variance thresholding function
valid_columns, x_train_variance_filtered = variance_thresholding(x_train_encoded_headers, x_train_mean_imputed)

# Update headers to match the filtered dataset
x_train_variance_filtered_headers = [x_train_encoded_headers[i] for i in valid_columns]


Step 2: Variance Thresholding - Reason: Low variance:
Removed 101 features: ['GENHLTH_7.0_encoded', 'GENHLTH_9.0_encoded', 'HLTHPLN1_7.0_encoded', 'HLTHPLN1_9.0_encoded', 'PERSDOC2_7.0_encoded', 'PERSDOC2_9.0_encoded', 'MEDCOST_7.0_encoded', 'MEDCOST_9.0_encoded', 'CHECKUP1_8.0_encoded', 'CHECKUP1_9.0_encoded', 'BPHIGH4_2.0_encoded', 'BPHIGH4_4.0_encoded', 'BPHIGH4_7.0_encoded', 'BPHIGH4_9.0_encoded', 'BLOODCHO_9.0_encoded', 'CHOLCHK_9.0_encoded', 'TOLDHI2_7.0_encoded', 'TOLDHI2_9.0_encoded', 'CVDSTRK3_7.0_encoded', 'CVDSTRK3_9.0_encoded', 'ASTHMA3_7.0_encoded', 'ASTHMA3_9.0_encoded', 'CHCSCNCR_7.0_encoded', 'CHCSCNCR_9.0_encoded', 'CHCOCNCR_7.0_encoded', 'CHCOCNCR_9.0_encoded', 'CHCCOPD1_7.0_encoded', 'CHCCOPD1_9.0_encoded', 'HAVARTH3_7.0_encoded', 'HAVARTH3_9.0_encoded', 'ADDEPEV2_7.0_encoded', 'ADDEPEV2_9.0_encoded', 'CHCKIDNY_7.0_encoded', 'CHCKIDNY_9.0_encoded', 'DIABETE3_2.0_encoded', 'DIABETE3_7.0_encoded', 'DIABETE3_9.0_encoded', 'MARITAL_9.0_encoded', 'EDUCA_1.0_encoded', 'ED

## Step 3 : Correlation analysis

In [73]:
def correlation_analysis(headers, x_data, y_data, low_threshold=0.05, high_threshold=0.9):
    """
    Removes features that have low correlation with the outcome variable (y_data),
    or that are highly correlated with other features.

    Args:
        headers (list): The list of feature names.
        x_data (np.array): The feature dataset (numpy array) with features as columns.
        y_data (np.array): The outcome variable (labels) to correlate with.
        low_threshold (float): Minimum Pearson correlation threshold (default is 0.05).
        high_threshold (float): Maximum Pearson correlation threshold (default is 0.9).

    Returns:
        tuple: A tuple containing:
            - valid_columns (list): The indices of the features that were kept.
            - filtered_data (np.array): The filtered dataset with only the valid columns.
    """
    valid_columns = []
    removed_columns = []

    # Ensure feature and outcome have the same non-NaN indices
    for i in range(x_data.shape[1]):
        feature = x_data[:, i]

        # Create a mask where both feature and y_data are non-NaN
        non_nan_mask = ~np.isnan(feature) & ~np.isnan(y_data)

        # Apply mask to both feature and y_data
        feature_clean = feature[non_nan_mask]
        y_data_clean = y_data[non_nan_mask]

        # Compute correlation if both feature_clean and y_data_clean have enough data
        if len(feature_clean) > 1 and len(y_data_clean) > 1:
            correlation = np.corrcoef(feature_clean, y_data_clean)[0, 1]

            # Check if the correlation is within the desired range
            if abs(correlation) >= low_threshold and abs(correlation) <= high_threshold:
                valid_columns.append(i)
            else:
                removed_columns.append(i)
        else:
            removed_columns.append(i)

    # Log removed features
    log_and_print_removals(headers, removed_columns, "3: Correlation Analysis", "Low or high correlation")

    # Store the removed features
    removed_features_names = [headers[i] for i in removed_columns]
    removed_correlation.extend(removed_features_names)

    # Filter valid columns
    filtered_data = x_data[:, valid_columns]
    return valid_columns, filtered_data

In [74]:
# Apply the correlation analysis function
valid_columns, x_train_correlation_filtered = correlation_analysis(x_train_variance_filtered_headers, x_train_variance_filtered, y_train)

# Update headers to match the filtered dataset
x_train_correlation_filtered_headers = [x_train_variance_filtered_headers[i] for i in valid_columns]


Step 3: Correlation Analysis - Reason: Low or high correlation:
Removed 224 features: ['_STATE', 'FMONTH', 'IDATE', 'IMONTH', 'IDAY', 'IYEAR_2015.0_encoded', 'IYEAR_2016.0_encoded', 'DISPCODE_1100.0_encoded', 'DISPCODE_1200.0_encoded', 'GENHLTH_3.0_encoded', 'MENTHLTH', 'HLTHPLN1_1.0_encoded', 'HLTHPLN1_2.0_encoded', 'PERSDOC2_1.0_encoded', 'MEDCOST_1.0_encoded', 'MEDCOST_2.0_encoded', 'CHECKUP1_2.0_encoded', 'CHECKUP1_3.0_encoded', 'CHECKUP1_4.0_encoded', 'CHECKUP1_7.0_encoded', 'BLOODCHO_7.0_encoded', 'CHOLCHK_4.0_encoded', 'CHOLCHK_7.0_encoded', 'ASTHMA3_1.0_encoded', 'DIABETE3_4.0_encoded', 'MARITAL_1.0_encoded', 'MARITAL_2.0_encoded', 'MARITAL_4.0_encoded', 'MARITAL_6.0_encoded', 'EDUCA_2.0_encoded', 'EDUCA_3.0_encoded', 'EDUCA_4.0_encoded', 'EDUCA_5.0_encoded', 'RENTHOM1_1.0_encoded', 'RENTHOM1_2.0_encoded', 'RENTHOM1_3.0_encoded', 'EMPLOY1_2.0_encoded', 'EMPLOY1_3.0_encoded', 'EMPLOY1_4.0_encoded', 'EMPLOY1_5.0_encoded', 'EMPLOY1_6.0_encoded', 'INCOME2_1.0_encoded', 'INCOME2_3.

In [77]:
print(len(x_train_correlation_filtered_headers))

138


### D. Statistical test

In [82]:
def statistical_test_analysis(headers, x_data, y_data, p_value_threshold=0.05):
    """
    Retains features that have a statistically significant association with the outcome variable (y_data),
    using an F-test for continuous features and a Chi-Square test for categorical features.

    Args:
        headers (list): The list of feature names.
        x_data (np.array): The feature dataset (numpy array) with features as columns.
        y_data (np.array): The outcome variable (labels) to test associations with.
        p_value_threshold (float): The significance level threshold for retaining features (default is 0.05).

    Returns:
        tuple: A tuple containing:
            - valid_columns (list): The indices of the features that were kept.
            - filtered_data (np.array): The filtered dataset with only the valid columns.
    """
    valid_columns = []
    removed_columns = []

    # Iterate over each feature
    for i in range(x_data.shape[1]):
        feature = x_data[:, i]
        
        # Create a mask where both feature and y_data are non-NaN
        non_nan_mask = ~np.isnan(feature) & ~np.isnan(y_data)
        feature_clean = feature[non_nan_mask]
        y_data_clean = y_data[non_nan_mask]

        # Check if the feature is continuous or categorical
        unique_vals = np.unique(feature_clean)
        if len(unique_vals) > 10:  # Assume continuous if more than 10 unique values
            # Perform F-test for continuous features
            classes = [feature_clean[y_data_clean == label] for label in np.unique(y_data_clean)]
            overall_var = np.var(feature_clean)
            between_group_var = np.var([np.mean(cls) for cls in classes])
            f_statistic = between_group_var / overall_var if overall_var != 0 else 0
            
            # Calculate p-value approximation based on F-statistic
            p_value = 1 - (1 / (1 + f_statistic))  # Simplified approximation

        else:
            # Perform Chi-Square test for categorical features
            unique_y = np.unique(y_data_clean)
            contingency_table = np.array([[(feature_clean == val).sum() & (y_data_clean == label).sum() for label in unique_y] for val in unique_vals])
            chi_square_statistic = np.sum((contingency_table - np.mean(contingency_table))**2 / np.mean(contingency_table))
            p_value = 1 - (1 / (1 + chi_square_statistic))  # Simplified approximation

        # Determine if the feature is statistically significant
        if p_value < p_value_threshold:
            valid_columns.append(i)
        else:
            removed_columns.append(i)

    # Log removed features
    log_and_print_removals(headers, removed_columns, "4: Statistical Test Analysis", "Not statistically significant")

    # Filter valid columns
    filtered_data = x_data[:, valid_columns]
    return valid_columns, filtered_data

In [85]:
print(len(x_train_variance_filtered_headers))

362


In [86]:
# Applying the statistical test analysis to the dataset after correlation analysis
removed_statistical = []  # List to store names of removed statistically insignificant features
valid_columns, x_train_statistical_filtered = statistical_test_analysis(x_train_variance_filtered_headers, x_train_variance_filtered, y_train)

# Update headers to match the filtered dataset
x_train_statistical_filtered_headers = [x_train_variance_filtered_headers[i] for i in valid_columns]

# Display results
print("Filtered dataset shape after statistical analysis:", x_train_statistical_filtered.shape)
print("Remaining features:", x_train_statistical_filtered_headers)
print("Removed features due to lack of statistical significance:", removed_statistical)


Step 4: Statistical Test Analysis - Reason: Not statistically significant:
Removed 320 features: ['IYEAR_2015.0_encoded', 'IYEAR_2016.0_encoded', 'DISPCODE_1100.0_encoded', 'DISPCODE_1200.0_encoded', 'GENHLTH_1.0_encoded', 'GENHLTH_2.0_encoded', 'GENHLTH_3.0_encoded', 'GENHLTH_4.0_encoded', 'GENHLTH_5.0_encoded', 'HLTHPLN1_1.0_encoded', 'HLTHPLN1_2.0_encoded', 'PERSDOC2_1.0_encoded', 'PERSDOC2_2.0_encoded', 'PERSDOC2_3.0_encoded', 'MEDCOST_1.0_encoded', 'MEDCOST_2.0_encoded', 'CHECKUP1_1.0_encoded', 'CHECKUP1_2.0_encoded', 'CHECKUP1_3.0_encoded', 'CHECKUP1_4.0_encoded', 'CHECKUP1_7.0_encoded', 'BPHIGH4_1.0_encoded', 'BPHIGH4_3.0_encoded', 'BLOODCHO_1.0_encoded', 'BLOODCHO_2.0_encoded', 'BLOODCHO_7.0_encoded', 'CHOLCHK_1.0_encoded', 'CHOLCHK_2.0_encoded', 'CHOLCHK_3.0_encoded', 'CHOLCHK_4.0_encoded', 'CHOLCHK_7.0_encoded', 'TOLDHI2_1.0_encoded', 'TOLDHI2_2.0_encoded', 'CVDSTRK3_1.0_encoded', 'CVDSTRK3_2.0_encoded', 'ASTHMA3_1.0_encoded', 'ASTHMA3_2.0_encoded', 'CHCSCNCR_1.0_encoded', '

In [87]:
print(len(x_train_statistical_filtered_headers))

42


In [75]:
# Print the final removal log for interpretability
print("\nRemoval Log:")
for entry in removal_log:
    print(entry)

print("Preprocessing complete. Cleaned dataset saved as 'cleaned_dataset.csv'.")


Removal Log:
CTELENUM removed at 1: Remove Missing Values: Too many missing values
PVTRESD1 removed at 1: Remove Missing Values: Too many missing values
COLGHOUS removed at 1: Remove Missing Values: Too many missing values
STATERES removed at 1: Remove Missing Values: Too many missing values
CELLFON3 removed at 1: Remove Missing Values: Too many missing values
LADULT removed at 1: Remove Missing Values: Too many missing values
NUMADULT removed at 1: Remove Missing Values: Too many missing values
NUMMEN removed at 1: Remove Missing Values: Too many missing values
NUMWOMEN removed at 1: Remove Missing Values: Too many missing values
CTELNUM1 removed at 1: Remove Missing Values: Too many missing values
CELLFON2 removed at 1: Remove Missing Values: Too many missing values
CADULT removed at 1: Remove Missing Values: Too many missing values
PVTRESD2 removed at 1: Remove Missing Values: Too many missing values
CCLGHOUS removed at 1: Remove Missing Values: Too many missing values
CSTATE remov

In [76]:
removed_features = removed_missing_values + removed_variance + removed_correlation
print(removed_features)


['CTELENUM', 'PVTRESD1', 'COLGHOUS', 'STATERES', 'CELLFON3', 'LADULT', 'NUMADULT', 'NUMMEN', 'NUMWOMEN', 'CTELNUM1', 'CELLFON2', 'CADULT', 'PVTRESD2', 'CCLGHOUS', 'CSTATE', 'LANDLINE', 'HHADULT', 'POORHLTH', 'BPMEDS', 'ASTHNOW', 'DIABAGE2', 'NUMHHOL2', 'NUMPHON2', 'CPDEMO1', 'PREGNANT', 'SMOKDAY2', 'STOPSMK2', 'LASTSMK2', 'AVEDRNK2', 'DRNK3GE5', 'MAXDRNKS', 'EXRACT11', 'EXEROFT1', 'EXERHMM1', 'EXRACT21', 'EXEROFT2', 'EXERHMM2', 'LMTJOIN3', 'ARTHDIS2', 'ARTHSOCL', 'JOINPAIN', 'FLSHTMY2', 'IMFVPLAC', 'HIVTSTD3', 'WHRTST10', 'PDIABTST', 'PREDIAB1', 'INSULIN', 'BLDSUGAR', 'FEETCHK2', 'DOCTDIAB', 'CHKHEMO3', 'FEETCHK', 'EYEEXAM', 'DIABEYE', 'DIABEDU', 'CAREGIV1', 'CRGVREL1', 'CRGVLNG1', 'CRGVHRS1', 'CRGVPRB1', 'CRGVPERS', 'CRGVHOUS', 'CRGVMST2', 'CRGVEXPT', 'VIDFCLT2', 'VIREDIF3', 'VIPRFVS2', 'VINOCRE2', 'VIEYEXM2', 'VIINSUR2', 'VICTRCT4', 'VIGLUMA2', 'VIMACDG2', 'CIMEMLOS', 'CDHOUSE', 'CDASSIST', 'CDHELP', 'CDSOCIAL', 'CDDISCUS', 'WTCHSALT', 'LONGWTCH', 'DRADVISE', 'ASTHMAGE', 'ASATTACK', 

## For us

In [115]:
import pandas as pd

# Load your Excel file with all features and their meanings
input_file_path = 'BRFSS_2015_Cleaned_Parameters_excel.xlsx'
df_parameters = pd.read_excel(input_file_path)

# List of conserved features to filter
conserved_features = x_train_headers

# Filter the DataFrame for conserved features
df_conserved = df_parameters[df_parameters['Parameter'].isin(conserved_features)]

# Save the filtered data to a new Excel file
output_file_path = 'conserved_features_output.xlsx'
df_conserved.to_excel(output_file_path, index=False)

print(f"Excel file generated and saved as {output_file_path}")

C:\Users\utilisateur\AppData\Local\Temp\ipykernel_17564\3861114181.py:15: UserWarning: Pandas requires version '3.0.5' or newer of 'xlsxwriter' (version '3.0.3' currently installed).
  df_conserved.to_excel(output_file_path, index=False)


Excel file generated and saved as conserved_features_output.xlsx


In [119]:
# Parse the removal log into a structured format
removed_features_ = []
removal_reasons = []

for log in removal_log:
    feature, reason = log.split(" removed at ")[0], log.split(": ", 1)[1]
    removed_features_.append(feature)
    removal_reasons.append(reason)

# Filter the DataFrame to include only the removed features
df_removed_info = df_parameters[df_parameters['Parameter'].isin(removed_features_)]

# Add the reason for removal to the DataFrame
df_removed_info['Reason of Removal'] = df_removed_info['Parameter'].map(dict(zip(removed_features_, removal_reasons)))

# Save the updated DataFrame with feature names, descriptions, value meanings, and reasons for removal to an Excel file
output_file_path = 'removed_features_with_description_log.xlsx'
df_removed_info.to_excel(output_file_path, index=False)

print(f"Excel file generated and saved as {output_file_path}")

Excel file generated and saved as removed_features_with_description_log.xlsx


C:\Users\utilisateur\AppData\Local\Temp\ipykernel_17564\2870947207.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_removed_info['Reason of Removal'] = df_removed_info['Parameter'].map(dict(zip(removed_features_, removal_reasons)))
C:\Users\utilisateur\AppData\Local\Temp\ipykernel_17564\2870947207.py:18: UserWarning: Pandas requires version '3.0.5' or newer of 'xlsxwriter' (version '3.0.3' currently installed).
  df_removed_info.to_excel(output_file_path, index=False)
